# Project: Cardiovascular Disease: Causes, Treatment, and Prevention

In this project, I will investigate one of the the major causes of death in the world: cardiovascular disease!

In [4]:
# Run this cell to set up the notebook, but please don't change it.
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
np.set_printoptions(legacy='1.13')

import d8error